<a href="https://colab.research.google.com/github/fasilvak10/bmo-appscript-google-version/blob/main/2022_BonosLEC_Unificado_902_sinArt9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>================ BONOS LEC - UNIFICADO (con Bono 902) =================</h1>

--- Se importan las Librerías utilizadas en el código y se especifica la ruta del archivo XLS a procesar (en este caso desde el Drive).

In [ ]:
pip install xlsxwriter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import pandas as pd
import time
from datetime import date
from datetime import datetime
from pytz import timezone  
from dateutil.relativedelta import relativedelta

In [ ]:
ruta_archivo_mectra = "/content/drive/My Drive/BonosLEC/Archivos procesamiento/Mectra/lps_202212.csv"
ruta_archivo_bonos_insumo = "/content/drive/My Drive/BonosLEC/Archivos procesamiento/Unificado/202212 Exportaciones Acto Ajuste.xlsx"
ruta_archivo_insumo_empresas = "/content/drive/My Drive/BonosLEC/Archivos procesamiento/Unificado/Empresas_Tipo.xlsx"

tope_emp = 1000000000
factor_tope = 1

fecha_generacion = datetime.now().strftime("%Y") + datetime.now().strftime("%m") + datetime.now().strftime("%d")
anio_generacion = datetime.now().strftime("%Y")
argentina = timezone('America/Argentina/Buenos_Aires')
hora_generacion = datetime.now(argentina).strftime('%H%M%S')

numero_verificador_900 = 994    # Número verificador que tendra el archivo TXT
numero_certificado_900 = 11923  # Ultimo número de certificado emitido para el Bono 900, se irá incrementando en cada registro 02

numero_verificador_901 = 922    # Número verificador que tendra el archivo TXT
numero_certificado_901 = 7852   # Ultimo número de certificado emitido para el Bono 901, se irá incrementando en cada registro 02

numero_verificador_902 = 1      # Número verificador que tendra el archivo TXT
numero_certificado_902 = 0      # Ultimo número de certificado emitido para el Bono 902, se irá incrementando en cada registro 02

## Agregado Reporte General

agrega_reporte_general = "Si"
nro_cert_900_inicial = numero_certificado_900 + 1
nro_cert_901_inicial = numero_certificado_901 + 1
nro_cert_902_inicial = numero_certificado_902 + 1

--- Se leen los archivos correspondiente a Mectra, Bonos Insumos y Tipos de Empresas. 

--- Se renombran y formatean algunas columnas en los DFs generados.

In [ ]:
mectra = pd.read_csv(ruta_archivo_mectra, sep=',')
mectra = mectra.round(2)
mectra = mectra.drop(columns=['zona','codprov'])

insumo = pd.read_excel(ruta_archivo_bonos_insumo, sheet_name="Hoja1")
insumo.rename(columns={"CUIT": "cuit", "Razon Social": "razonsocial", "Exportacion": "exportacion", "Fecha de Acto": "fechaacto", "Ajuste": "ajuste"}, inplace=True)
insumo["exportacion"] = insumo["exportacion"].str.replace("%","").str.replace(",",".").astype(np.float)
insumo["ajuste"] = insumo["ajuste"].str.replace(",",".").astype(np.float)
insumo = insumo.round(2)

empresas = pd.read_excel(ruta_archivo_insumo_empresas, sheet_name="Hoja1")
empresas.rename(columns={"CUIT": "cuit", "Razon Social": "razonsocial", "Tipo": "tipo", "Código": "codigo"}, inplace=True)

In [ ]:
insumo

In [ ]:
mectra

--- Solo nos quedamos con los CUITs de las empresas que nos interesan, las que estan presentes en Bonos Insumo.

--- Marcamos dentro de las empresas informadas en Insumo, cantidades para los nuevos códigos en Mectra y que tipo de empresa es (adherente o inscripión)

In [ ]:
insumo["codigo_empresa"] = ""
insumo["tipo_empresa"] = ""
insumo["total_lec"] = ""
insumo["total_lec_mectra_dj"] = ""
insumo["total_lec_emp_promo"] = ""
insumo["art9_lec_133"] = ""
insumo["promo_lec_132"] = ""
cuit_insumo = insumo['cuit'].tolist()

for x in range(len(mectra)):
  if mectra.loc[x, "cuit"] not in cuit_insumo:
    mectra = mectra.drop(x)

mectra.drop_duplicates()
mectra["index"] = [x for x in range(len(mectra))]
mectra.set_index("index", inplace=True)

for x in range(len(insumo)):
  insumo.loc[x, "codigo_empresa"] = empresas.loc[empresas["cuit"] == insumo.loc[x, "cuit"], "codigo"].item()
  insumo.loc[x, "tipo_empresa"] = empresas.loc[empresas["cuit"] == insumo.loc[x, "cuit"], "tipo"].item()
  insumo.loc[x, "total_lec"] = len(mectra.loc[mectra["cuit"] == insumo.loc[x, "cuit"]])
  insumo.loc[x, "promo_lec_132"] = len(mectra.loc[(mectra["cuit"] == insumo.loc[x, "cuit"]) & (mectra["act_trab"] == 132)])
  insumo.loc[x, "art9_lec_133"] = len(mectra.loc[(mectra["cuit"] == insumo.loc[x, "cuit"]) & (mectra["act_trab"] == 133)])
  insumo.loc[x, "total_lec_emp_promo"] = insumo.loc[x, "promo_lec_132"] + insumo.loc[x, "art9_lec_133"]

--- Se calcula el monto ANSSAL a descontar de las contribuciones de los empleados.

--- Si el monto de Conss menos el valor calculado de Anssal da menor a 1, el mismo queda en 0 (para evitar valores negativos).



In [ ]:
mectra["conos_mayor_cero"] = ""
mectra["rem8_menor_2400"] = ""
mectra["porc_anssal_os"] = ""
mectra["monto_anssal"] = ""
mectra["conss_sin_anssal"] = ""

for x in range(len(mectra)):
  if mectra.loc[x, "conos"] > 0:
    mectra.loc[x, "conos_mayor_cero"] = 1
  else:
    mectra.loc[x, "conos_mayor_cero"] = 0

  if mectra.loc[x, "rem8"] < 2400:
    mectra.loc[x, "rem8_menor_2400"] = 0.05
  else:
    mectra.loc[x, "rem8_menor_2400"] = 0

  if (str(mectra.loc[x, "cod_bosocial"])[:1] == '4' and mectra.loc[x, "cod_bosocial"] > 100000):
    mectra.loc[x, "porc_anssal_os"] = 0.2 - mectra.loc[x, "rem8_menor_2400"]
  else:
    mectra.loc[x, "porc_anssal_os"] = 0.15 - mectra.loc[x, "rem8_menor_2400"]
  
  mectra.loc[x, "monto_anssal"] = (mectra.loc[x, "rem8"]*0.06 + mectra.loc[x, "imp_ad_os"])*mectra.loc[x, "conos_mayor_cero"]*mectra.loc[x, "porc_anssal_os"]
  mectra.loc[x, "conss_sin_anssal"] = mectra.loc[x, "conss"] - mectra.loc[x, "monto_anssal"]

  if mectra.loc[x, "conss_sin_anssal"] < 1:
    mectra.loc[x, "conss_sin_anssal"] = 0

mectra = mectra.round(2)

--- Se calcula tambien el monto aportado a los bonos en base a las contribuciones de cada empleado (se diferencia el aporte extra de Art9).

In [ ]:
mectra["conss_sin_anssal_art8"] = ""
mectra["conss_sin_anssal_art9"] = ""
periodo_mectra = np.unique(mectra['mes']).tolist()

for x in range(len(mectra)):
  if mectra.loc[x, "act_trab"] == 132:
    mectra.loc[x, "conss_sin_anssal_art8"] = mectra.loc[x, "conss_sin_anssal"]*0.7
    mectra.loc[x, "conss_sin_anssal_art9"] = 0
  elif mectra.loc[x, "act_trab"] == 133:
    mectra.loc[x, "conss_sin_anssal_art8"] = mectra.loc[x, "conss_sin_anssal"]*0.7
    mectra.loc[x, "conss_sin_anssal_art9"] = mectra.loc[x, "conss_sin_anssal"]*0.1
  else:
    mectra.loc[x, "conss_sin_anssal_art8"] = 0
    mectra.loc[x, "conss_sin_anssal_art9"] = 0

--- Se realiza el cálculo del monto de los bonos para cada una de las empresas informadas en insumo.

--- También se realizan otros calculos a modo de reporte.

In [ ]:
insumo["contrib_total_lec"] = ""
insumo["contrib_promo_art8"] = ""
insumo["contrib_promo_art9"] = ""
insumo["total_bonos_sin_ajuste"] = ""
insumo["total_bonos"] = ""
insumo["total_bono_900"] = ""
insumo["total_bono_901"] = ""
insumo["total_bono_902"] = ""
insumo["fecha_inicio_inscripcion"] = ""
insumo["fecha_inicio"] = ""
insumo["fecha_fin"] = ""
cuit_insumo = insumo['cuit'].tolist()

for x in range(len(insumo)):
  cuit_insumo = insumo.loc[x, "cuit"]
  cant_promo = insumo.loc[x, "promo_lec_132"] + insumo.loc[x, "art9_lec_133"]
  if cant_promo > tope_emp:
    factor_tope = tope_emp/cant_promo
  else:
    factor_tope = 1
    
  insumo.loc[x, "contrib_total_lec"] = mectra.loc[mectra["cuit"] == cuit_insumo, "conss_sin_anssal"].sum()
  insumo.loc[x, "contrib_promo_art8"] = mectra.loc[mectra["cuit"] == cuit_insumo, "conss_sin_anssal_art8"].sum()
  insumo.loc[x, "contrib_promo_art8"] = insumo.loc[x, "contrib_promo_art8"] * factor_tope
  insumo.loc[x, "contrib_promo_art9"] = mectra.loc[mectra["cuit"] == cuit_insumo, "conss_sin_anssal_art9"].sum()
  insumo.loc[x, "contrib_promo_art9"] = insumo.loc[x, "contrib_promo_art9"] * factor_tope
  insumo.loc[x, "total_bonos_sin_ajuste"] = insumo.loc[x, "contrib_promo_art8"]             ## porque aun no se otorga el beneficio de Art9
  if insumo.loc[x, "total_bonos_sin_ajuste"] > insumo.loc[x, "ajuste"]:
    insumo.loc[x, "total_bonos"] = insumo.loc[x, "total_bonos_sin_ajuste"] - insumo.loc[x, "ajuste"]
  else:
    insumo.loc[x, "total_bonos"] = 0
  if insumo.loc[x, "exportacion"] < 70 or periodo_mectra[0] < 202210:
    insumo.loc[x, "total_bono_901"] = insumo.loc[x, "total_bonos"] * (insumo.loc[x, "exportacion"]/100)
    insumo.loc[x, "total_bono_902"] = 0
  else:
    insumo.loc[x, "total_bono_902"] = insumo.loc[x, "total_bonos"] * (insumo.loc[x, "exportacion"]/100)
    insumo.loc[x, "total_bono_901"] = 0
  insumo.loc[x, "total_bono_900"] = insumo.loc[x, "total_bonos"] - (insumo.loc[x, "total_bono_901"] + insumo.loc[x, "total_bono_902"])
  if insumo.loc[x, "codigo_empresa"] == 2:
    insumo.loc[x, "fecha_inicio_inscripcion"] = (insumo.loc[x, "fechaacto"] + relativedelta(months = 1)).replace(day=1)
  else:
    insumo.loc[x, "fecha_inicio_inscripcion"] = "20200101"
  insumo.loc[x, "fecha_inicio"] = date.today().isoformat()
  insumo.loc[x, "fecha_fin"] = date.today() + relativedelta(years = 2) + relativedelta(days = -1)

insumo = insumo.round(2)

--- Se genera el Bono 900

In [ ]:
bono_900 = insumo.loc[(insumo["total_bono_900"] > 0)]
bono_900["index"] = [x for x in range(len(bono_900))]
bono_900.set_index("index", inplace=True)

cant_bonos = len(bono_900)
for x in range(cant_bonos):
  bono_900.loc[x ,"total_bono_900"] = bono_900.loc[x ,"total_bono_900"].round(2)
monto_total_bonos = "{:.2f}".format(bono_900["total_bono_900"].sum())

registro01 = "01"                                          # Tipo de Registro
registro01 += "30710817452"                                # CUIT Ministerio
registro01 += "00"                                         # Establecimiento
registro01 += "083"                                        # Impuesto
registro01 += "019"                                        # Concepto
registro01 += str(periodo_mectra[0])                       # Período
registro01 += "0"                                          # Código de Rectificativa
registro01 += str(numero_verificador_900).zfill(6)         # Número Verificador - ES UN VALOR INTERNO DE IDENTIFICACIÓN QUE UD LE QUIERAN COLOCAR
registro01 += "1400"                                       # Número de Formulario
registro01 += "00100"                                      # Versión
registro01 += hora_generacion                              # Hora - generación del archivo HHMMSS
registro01 += "0100000"                                    # Versión del exe
registro01 += str(cant_bonos).zfill(7)                     # Cantidad de Bonos
registro01 += monto_total_bonos.replace(".","").zfill(15)  # Monto total Bonos con dos decimales sin coma (13 valores mas 2 dígitos)
lst_bono_900 = [registro01]

for x in range(cant_bonos):
  numero_certificado_900 += 1
  registro02 = "02"                                                                           # Tipo de Registro
  registro02 += "900"                                                                         # Tipo de Certificado - 900 / 901
  registro02 += anio_generacion                                                               # Año de emisión de Certificado
  registro02 += str(numero_certificado_900).zfill(9)                                          # Número de Certificado - ESTE NRO ES A CRITERIO DE CVSA. DEBE SER CORRELATIVO Y CADA AÑO REINICIA LA NUMERACIÓN
  registro02 += str(bono_900.loc[x ,"cuit"])                                                  # CUIT de la empresa beneficiaria
  registro02 += "{:.2f}".format(bono_900.loc[x ,"total_bono_900"]).replace(".","").zfill(15)  # Monto con dos decimales y coma (13 valores mas 2 dígitos)
  registro02 += str(bono_900.loc[x ,"fechaacto"]).replace("-","")[:8]                         # Fecha de Expediente - Formato AAAAMMDD - ES UN DATO INTERNO DE CVSA. GENERALMENTE LOS ORGANISMOS EXTERNOS INFORMAN NRO DE EXPEDIENTE O CARPETA QUE DIO ORIGEN A ESE OTORGAMIENTO DEL BENEFICIO DEL CRÉDITO FISCAL. NO VALIDAMOS ESTE DATO.
  registro02 += str(bono_900.loc[x ,"fecha_inicio_inscripcion"]).replace("-","")[:8]          # Fecha desde - inscripcion en el registro (es la fecha de la emisión, que puede diferir de la fecha en el Registro)
  registro02 += str(bono_900.loc[x ,"fecha_fin"]).replace("-","")[:8]                         # Fecha hasta - máxima a utilizar el bono
  registro02 += "1"                                                                           # Estado
  lst_bono_900.append(registro02)

registro90 = "90"                                               # Tipo de Registro
registro90 += "010"                                             # Código - Aca irá el código 10 y en el campo Importe la Cantidad
registro90 += str(cant_bonos).zfill(15)                         # Importe - Cantidad de bonos informados
lst_bono_900.append(registro90)

registro90 = "90"                                               # Tipo de Registro
registro90 += "020"                                             # Código - en un segundo registro va 20 y en campo Importe va el Monto Total de Certificados
registro90 += monto_total_bonos.replace(".","").zfill(15)       # Importe - Monto Total de los Bonos informados
lst_bono_900.append(registro90)

--- Se genera el Bono 901

In [ ]:
bono_901 = insumo.loc[(insumo["total_bono_901"] > 0)]
bono_901["index"] = [x for x in range(len(bono_901))]
bono_901.set_index("index", inplace=True)

cant_bonos = len(bono_901)
for x in range(cant_bonos):
  bono_901.loc[x ,"total_bono_901"] = bono_901.loc[x ,"total_bono_901"].round(2)
monto_total_bonos = "{:.2f}".format(bono_901["total_bono_901"].sum())

registro01 = "01"                                          # Tipo de Registro
registro01 += "30710817452"                                # CUIT Ministerio
registro01 += "00"                                         # Establecimiento
registro01 += "083"                                        # Impuesto
registro01 += "019"                                        # Concepto
registro01 += str(periodo_mectra[0])                       # Período
registro01 += "0"                                          # Código de Rectificativa
registro01 += str(numero_verificador_901).zfill(6)         # Número Verificador - ES UN VALOR INTERNO DE IDENTIFICACIÓN QUE UD LE QUIERAN COLOCAR
registro01 += "1400"                                       # Número de Formulario
registro01 += "00100"                                      # Versión
registro01 += hora_generacion                              # Hora - generación del archivo HHMMSS
registro01 += "0100000"                                    # Versión del exe
registro01 += str(cant_bonos).zfill(7)                     # Cantidad de Bonos
registro01 += monto_total_bonos.replace(".","").zfill(15)  # Monto total Bonos con dos decimales sin coma (13 valores mas 2 dígitos)
lst_bono_901 = [registro01]

for x in range(len(bono_901)):
  numero_certificado_901 += 1
  registro02 = "02"                                                                           # Tipo de Registro
  registro02 += "901"                                                                         # Tipo de Certificado - 900 / 901
  registro02 += anio_generacion                                                               # Año de emisión de Certificado
  registro02 += str(numero_certificado_901).zfill(9)                                          # Número de Certificado - ESTE NRO ES A CRITERIO DE CVSA. DEBE SER CORRELATIVO Y CADA AÑO REINICIA LA NUMERACIÓN
  registro02 += str(bono_901.loc[x ,"cuit"])                                                  # CUIT de la empresa beneficiaria
  registro02 += "{:.2f}".format(bono_901.loc[x ,"total_bono_901"]).replace(".","").zfill(15)  # Monto con dos decimales y coma (13 valores mas 2 dígitos)
  registro02 += str(bono_901.loc[x ,"fechaacto"]).replace("-","")[:8]                         # Fecha de Expediente - Formato AAAAMMDD - ES UN DATO INTERNO DE CVSA. GENERALMENTE LOS ORGANISMOS EXTERNOS INFORMAN NRO DE EXPEDIENTE O CARPETA QUE DIO ORIGEN A ESE OTORGAMIENTO DEL BENEFICIO DEL CRÉDITO FISCAL. NO VALIDAMOS ESTE DATO.
  registro02 += str(bono_901.loc[x ,"fecha_inicio_inscripcion"]).replace("-","")[:8]          # Fecha desde - inscripcion en el registro (es la fecha de la emisión, que puede diferir de la fecha en el Registro)
  registro02 += str(bono_901.loc[x ,"fecha_fin"]).replace("-","")[:8]                         # Fecha hasta - máxima a utilizar el bono
  registro02 += "1"                                                                           # Estado
  lst_bono_901.append(registro02)

registro90 = "90"                                               # Tipo de Registro
registro90 += "010"                                             # Código - Aca irá el código 10 y en el campo Importe la Cantidad
registro90 += str(cant_bonos).zfill(15)                         # Importe - Cantidad de bonos informados
lst_bono_901.append(registro90)

registro90 = "90"                                               # Tipo de Registro
registro90 += "020"                                             # Código - en un segundo registro va 20 y en campo Importe va el Monto Total de Certificados
registro90 += monto_total_bonos.replace(".","").zfill(15)       # Importe - Monto Total de los Bonos informados
lst_bono_901.append(registro90)

--- Se genera el Bono 902

In [ ]:
if periodo_mectra[0] >= 202210:
  bono_902 = insumo.loc[(insumo["total_bono_902"] > 0)]
  bono_902["index"] = [x for x in range(len(bono_902))]
  bono_902.set_index("index", inplace=True)

  cant_bonos = len(bono_902)
  for x in range(cant_bonos):
    bono_902.loc[x ,"total_bono_902"] = bono_902.loc[x ,"total_bono_902"].round(2)
  monto_total_bonos = "{:.2f}".format(bono_902["total_bono_902"].sum())

  registro01 = "01"                                          # Tipo de Registro
  registro01 += "30710817452"                                # CUIT Ministerio
  registro01 += "00"                                         # Establecimiento
  registro01 += "083"                                        # Impuesto
  registro01 += "019"                                        # Concepto
  registro01 += str(periodo_mectra[0])                       # Período
  registro01 += "0"                                          # Código de Rectificativa
  registro01 += str(numero_verificador_902).zfill(6)         # Número Verificador - ES UN VALOR INTERNO DE IDENTIFICACIÓN QUE UD LE QUIERAN COLOCAR
  registro01 += "1400"                                       # Número de Formulario
  registro01 += "00100"                                      # Versión
  registro01 += hora_generacion                              # Hora - generación del archivo HHMMSS
  registro01 += "0100000"                                    # Versión del exe
  registro01 += str(cant_bonos).zfill(7)                     # Cantidad de Bonos
  registro01 += monto_total_bonos.replace(".","").zfill(15)  # Monto total Bonos con dos decimales sin coma (13 valores mas 2 dígitos)
  lst_bono_902 = [registro01]

  for x in range(len(bono_902)):
    numero_certificado_902 += 1
    registro02 = "02"                                                                           # Tipo de Registro
    registro02 += "902"                                                                         # Tipo de Certificado - 900 / 901
    registro02 += anio_generacion                                                               # Año de emisión de Certificado
    registro02 += str(numero_certificado_902).zfill(9)                                          # Número de Certificado - ESTE NRO ES A CRITERIO DE CVSA. DEBE SER CORRELATIVO Y CADA AÑO REINICIA LA NUMERACIÓN
    registro02 += str(bono_902.loc[x ,"cuit"])                                                  # CUIT de la empresa beneficiaria
    registro02 += "{:.2f}".format(bono_902.loc[x ,"total_bono_902"]).replace(".","").zfill(15)  # Monto con dos decimales y coma (13 valores mas 2 dígitos)
    registro02 += str(bono_902.loc[x ,"fechaacto"]).replace("-","")[:8]                         # Fecha de Expediente - Formato AAAAMMDD - ES UN DATO INTERNO DE CVSA. GENERALMENTE LOS ORGANISMOS EXTERNOS INFORMAN NRO DE EXPEDIENTE O CARPETA QUE DIO ORIGEN A ESE OTORGAMIENTO DEL BENEFICIO DEL CRÉDITO FISCAL. NO VALIDAMOS ESTE DATO.
    registro02 += str(bono_902.loc[x ,"fecha_inicio_inscripcion"]).replace("-","")[:8]          # Fecha desde - inscripcion en el registro (es la fecha de la emisión, que puede diferir de la fecha en el Registro)
    registro02 += str(bono_902.loc[x ,"fecha_fin"]).replace("-","")[:8]                         # Fecha hasta - máxima a utilizar el bono
    registro02 += "1"                                                                           # Estado
    lst_bono_902.append(registro02)

  registro90 = "90"                                               # Tipo de Registro
  registro90 += "010"                                             # Código - Aca irá el código 10 y en el campo Importe la Cantidad
  registro90 += str(cant_bonos).zfill(15)                         # Importe - Cantidad de bonos informados
  lst_bono_902.append(registro90)

  registro90 = "90"                                               # Tipo de Registro
  registro90 += "020"                                             # Código - en un segundo registro va 20 y en campo Importe va el Monto Total de Certificados
  registro90 += monto_total_bonos.replace(".","").zfill(15)       # Importe - Monto Total de los Bonos informados
  lst_bono_902.append(registro90)

--- Se imprimen ambos bonos en una archivo TXT para la presentación a AFIP.

In [ ]:
fecha = datetime.now().strftime("%Y") + datetime.now().strftime("%m") + datetime.now().strftime("%d")

if len(bono_900) >= 1:
  nombre_archivo_900 = "F1400.30710817452." + fecha +"."+ str(numero_verificador_900).zfill(4) +".txt"
  numpy_array_900 = pd.DataFrame(lst_bono_900).to_numpy()
  np.savetxt("/content/drive/MyDrive/BonosLEC/Archivos Bonos LEC/Bono900/Unificado/" + nombre_archivo_900, numpy_array_900, fmt = '%s')

if len(bono_901) >= 1:
  nombre_archivo_901 = "F1400.30710817452." + fecha +"."+ str(numero_verificador_901).zfill(4) +".txt"
  numpy_array_901 = pd.DataFrame(lst_bono_901).to_numpy()
  np.savetxt("/content/drive/MyDrive/BonosLEC/Archivos Bonos LEC/Bono901/Unificado/" + nombre_archivo_901, numpy_array_901, fmt = '%s')

if periodo_mectra[0] >= 202210:
  if len(bono_902) >= 1:
    nombre_archivo_902 = "F1400.30710817452." + fecha +"."+ str(numero_verificador_902).zfill(4) +".txt"
    numpy_array_902 = pd.DataFrame(lst_bono_902).to_numpy()
    np.savetxt("/content/drive/MyDrive/BonosLEC/Archivos Bonos LEC/Bono902/Unificado/" + nombre_archivo_902, numpy_array_902, fmt = '%s')

--- Se renombran las columnas para su exportación a Excel.

In [ ]:
insumo = insumo.drop(columns=['fecha_inicio_inscripcion', 'codigo_empresa'])
insumo.rename(columns={"cuit": "CUIT", "razonsocial": "Razón Social", "exportacion": "Exportación", "fechaacto": "Fecha Acto/ Fecha de Inscripción", "ajuste": "Ajuste",
                       "tipo_empresa": "Tipo Empresa", "total_lec": "Total Empleados Mectra", "total_lec_mectra_dj": "Total Empleados DJ y Mectra", "total_lec_emp_promo": "Total Emp Promovidos",
                       "art9_lec_133": "Total Emp Art9", "promo_lec_132": "Total Emp Art8", "contrib_total_lec": "Contrib Total (Mectra)",
                       "contrib_promo_art8": "Bono Art8 (70%)", "contrib_promo_art9": "Bono Art9 (10%)",
                       "total_bonos_sin_ajuste": "Bono Total Emitido (BTE)(aun no se descontó ajuste)", "total_bonos": "Total bonos 900+901+902 (BTE-Ajuste)",
                       "total_bono_900": "Monto Bono 900", "total_bono_901": "Monto Bono 901", "total_bono_902": "Monto Bono 902",
                       "fecha_inicio": "Fecha Alta Bono", "fecha_fin": "Fecha Vencimiento Bono"}
, inplace=True)

--- Se agrega el DF insumo al Reporte General Excel.

In [ ]:
if agrega_reporte_general == "Si":
  insumo["Fecha Procesamiento"] = date.today().isoformat()
  insumo["Tipo Emisión"] = "Unificado"
  insumo["Período"] = periodo_mectra[0]
  insumo["Nro Verificador 900"] = ""
  insumo["Nro Certificado 900"] = ""
  insumo["Nro Verificador 901"] = ""
  insumo["Nro Certificado 901"] = ""
  insumo["Nro Verificador 902"] = ""
  insumo["Nro Certificado 902"] = ""

  for x in range(len(insumo)):
    if insumo.loc[x, "Monto Bono 900"] > 0:
      insumo.loc[x, "Nro Verificador 900"] = numero_verificador_900
      insumo.loc[x, "Nro Certificado 900"] = nro_cert_900_inicial
      nro_cert_900_inicial = nro_cert_900_inicial + 1
    if insumo.loc[x, "Monto Bono 901"] > 0:
      insumo.loc[x, "Nro Verificador 901"] = numero_verificador_901
      insumo.loc[x, "Nro Certificado 901"] = nro_cert_901_inicial
      nro_cert_901_inicial = nro_cert_901_inicial + 1
    if insumo.loc[x, "Monto Bono 902"] > 0:
      insumo.loc[x, "Nro Verificador 902"] = numero_verificador_902
      insumo.loc[x, "Nro Certificado 902"] = nro_cert_902_inicial
      nro_cert_902_inicial = nro_cert_902_inicial + 1

  ruta_reporte_bonos_general = "/content/drive/My Drive/BonosLEC/Archivos Bonos LEC/Reporte/Reporte Emisión Bonos.xlsx"
  reporte_general = pd.read_excel(ruta_reporte_bonos_general, sheet_name="General")

  reporte_general = reporte_general.append(insumo)
  reporte_general.iloc[: , 1:]

  writer = pd.ExcelWriter(ruta_reporte_bonos_general, engine='xlsxwriter', date_format='YYYY-MM-DD', datetime_format='YYYY-MM-DD')
  reporte_general.to_excel(writer, sheet_name='General', startrow=1, header=False, index=False)

  workbook = writer.book
  worksheet = writer.sheets['General']
  (max_row, max_col) = reporte_general.shape
  column_settings = [{'header': column} for column in reporte_general.columns]
  worksheet.add_table(0, 0, max_row, max_col - 1, 
                     {'banded_rows': False, 'columns': column_settings,
                      'style': 'Table Style Light 11'})

  worksheet.set_column(0, max_col - 1, 14)
  writer.save()

  insumo = insumo.drop(columns=['Tipo Empresa','Fecha Procesamiento','Tipo Emisión','Período','Nro Verificador 900','Nro Certificado 900','Nro Verificador 901','Nro Certificado 901','Nro Verificador 902','Nro Certificado 902'])

--- Se exportan tanto el DF mectra como el de insumo con todos sus calculos a modo de reporte a Excel.


--- Se ordena la tabla con Contribución en forma descendente.

In [ ]:
insumo.sort_values(by=["CUIT"], ascending=False, inplace=True)

In [ ]:
nombre_reporte = "Reporte Bonos " + str(periodo_mectra[0]) + "_" + str(numero_verificador_900) + "_" + str(numero_verificador_901) +" - " + date.today().isoformat() + ".xlsx"
ruta_archivo_salida = "/content/drive/My Drive/BonosLEC/Archivos Bonos LEC/Reporte/Unificado/" + nombre_reporte

writer = pd.ExcelWriter(ruta_archivo_salida, engine='xlsxwriter',
                        date_format='YYYY-MM-DD', datetime_format='YYYY-MM-DD')
df = insumo
df.to_excel(writer, sheet_name='Sheet1', startrow=1, header=False, index=False)

# Get the xlsxwriter workbook and worksheet objects.
workbook = writer.book
worksheet = writer.sheets['Sheet1']

# Get the dimensions of the dataframe.
(max_row, max_col) = df.shape

# Create a list of column headers, to use in add_table().
column_settings = [{'header': column} for column in df.columns]

# Add the Excel table structure. Pandas will add the data.
worksheet.add_table(0, 0, max_row, max_col - 1, 
                    {'banded_rows': True, 'columns': column_settings,
                     'style': 'Table Style Light 11'})

# Make the columns wider for clarity.
worksheet.set_column(0, max_col - 1, 14)

# Close the Pandas Excel writer and output the Excel file.
writer.save()

In [ ]:
nombre_reporte_mectra = "Reporte Mectra " + str(periodo_mectra[0]) + "_" + str(numero_verificador_900) + "_" + str(numero_verificador_901) + "_" + str(numero_verificador_902) +" - " + date.today().isoformat() + ".xlsx"
ruta_archivo_salida_mectra = "/content/drive/My Drive/BonosLEC/Archivos Bonos LEC/Reporte/Unificado/" + nombre_reporte_mectra

writer = pd.ExcelWriter(ruta_archivo_salida_mectra, engine='xlsxwriter',
                        date_format='YYYY-MM-DD', datetime_format='YYYY-MM-DD')
df = mectra
df.to_excel(writer, sheet_name='Sheet1', startrow=1, header=False, index=False)

# Get the xlsxwriter workbook and worksheet objects.
workbook = writer.book
worksheet = writer.sheets['Sheet1']

# Get the dimensions of the dataframe.
(max_row, max_col) = df.shape

# Create a list of column headers, to use in add_table().
column_settings = [{'header': column} for column in df.columns]

# Add the Excel table structure. Pandas will add the data.
worksheet.add_table(0, 0, max_row, max_col - 1, 
                    {'banded_rows': True, 'columns': column_settings,
                     'style': 'Table Style Light 11'})

# Make the columns wider for clarity.
worksheet.set_column(0, max_col - 1, 14)

# Close the Pandas Excel writer and output the Excel file.
writer.save()

<h1>================== Fin del código ==================</h1>